In [1]:
import pandas as pd
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

In [2]:
cc = pd.read_csv('data\mabee2606_features_QF.csv', usecols=[0,1,7])
cc['type'] = ['NV' if 'NV' in name else 'Inc' if 'Inc' in name else 'Clamp' for name in cc['name']]
cc['type_noNV'] = ['Clamp' for _ in cc['name']]
cc['NV'] = [1 if 'NV' in name else 0 for name in cc['name']]
perfs = pd.read_csv('data\mabee2606_perfs_BIASED.csv', usecols=[0,1,7])
perfs['type'] = 'PERFS'
combined = pd.concat([cc.loc[cc['type'].isin(['Clamp', 'NV', 'Inc'])], perfs])
combined['AngleFromHighSideClockwiseDegrees'] = [-1*(angle-180) if angle > 180 else angle for angle in combined['perfAngleFromHighSideClockwiseDegrees']]

In [ ]:
pie = make_subplots(rows=1, cols=1, specs=[[{"type": "domain"}]])

pie.add_trace(go.Pie(
     values=cc.loc[cc['type_noNV'].isin(['Clamp'])]['type'].value_counts(),
     labels=cc.loc[cc['type_noNV'].isin(['Clamp'])]['type'].value_counts().index,
     domain=dict(x=[0, 0.5]),
     name="Clamp",
     sort=False,
     marker_colors = ['red', 'lightgrey', 'darkgrey']
     ), 
     row=1, col=1)

# pie.update_layout(height=600, width=800, title_text="Success rate by jewelry type", template="plotly")
pie.update_layout(title_text="Success rate by jewelry type", template="plotly")
pie.show()

In [ ]:
fig  = combined.plot(x='AngleFromHighSideClockwiseDegrees', y='depth', kind='scatter', color='type', custom_data=['name'], color_discrete_map={'Clamp':'red', 'PERFS':'blue', 'NV':'grey', 'Inc':'darkgrey'})
fig.update_layout(hovermode="y", title="Jewelry vs Perfs", legend_title="Type", legend_orientation="h", template="plotly")
fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-185, 185])
fig.update_yaxes(autorange="reversed")

In [ ]:
pio.write_html(fig, file='plotly/m.html', auto_open=False)
pio.write_html(pie, file='plotly/mc.html', auto_open=False)

Well 2

In [85]:
cc = pd.read_csv('data\Seidel Unit A3_features_QF.csv', usecols=[0,1,7])
cc['type'] = ['CrossCouple' if 'CC' in name else 'BlastProtector' if 'BP' in name else 'Centralizer' for name in cc['name']]
cc['NV'] = ['NV' if 'NV' in name else pd.NA for name in cc['name']]
cc.loc[cc['NV'] == 'NV', 'perfAngleFromHighSideClockwiseDegrees'] = pd.NA
cc['J'] = [name.split()[0] if 'J' in name.split()[0] else pd.NA for name in cc['name']]
cc['BP'] = [name.split()[1][:3] if 'BP' in name.split()[1] else pd.NA for name in cc['name']]
cc['CC'] = [name.split()[1][:-2] if 'CC' in name.split()[1] else pd.NA for name in cc['name']]
cc['C'] = [name.split()[1] if name.split()[1].startswith('C1') else pd.NA for name in cc['name']]


perfs = pd.read_csv('data\Seidel Unit A3_Perfs_BIASED.csv', usecols=[0,1,7])
perfs['type'] = 'PERFS'

In [86]:
features = pd.concat([cc[['depth', 'J', 'BP', 'perfAngleFromHighSideClockwiseDegrees']].groupby(['J','BP']).mean(numeric_only=False), cc[['depth', 'CC', 'perfAngleFromHighSideClockwiseDegrees']].groupby(['CC']).mean(numeric_only=False)])
features['name'] = features.index
features = features.sort_values(by=['depth'], ascending=False)
features.reset_index(drop=True, inplace=True)
features['type'] = ['BlastProtector' if 'BP' in name[1] else 'CrossCoupling' if 'CC' in name else 'Centralizer' for name in features['name']]
features['type'] = ['BlastProtector' if 'BP' in name[1] else 'CrossCoupling' if 'CC' in name else 'Centralizer' for name in features['name']]
features.loc[features['perfAngleFromHighSideClockwiseDegrees'].isna(), 'type'] = features.loc[features['perfAngleFromHighSideClockwiseDegrees'].isna(), 'type']+' NV'
features.interpolate(method='nearest', inplace=True)



combined = pd.concat([features, perfs])
combined['AngleFromHighSideClockwiseDegrees'] = [-1*(angle-180) if angle > 180 else angle for angle in combined['perfAngleFromHighSideClockwiseDegrees']]
combined['angle'] = combined['perfAngleFromHighSideClockwiseDegrees']//10*10

In [92]:
fig  = combined.plot(x='AngleFromHighSideClockwiseDegrees', y='depth', kind='scatter', color='type', custom_data=['name', 'angle'], color_discrete_map={'CrossCoupling':'lightsalmon', 'CrossCoupling NV':'lightsalmon', 'BlastProtector':'crimson', 'BlastProtector NV':'crimson', 'Centralizer':'lightblue', 'PERFS':'cornflowerblue', 'NV':'grey'})
fig.update_layout(hovermode="y", title="Jewelry vs Perfs", legend_title="Type", legend_orientation="h", template="plotly")
fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-185, 185])
fig.update_yaxes(autorange="reversed")

In [93]:
features['type'].value_counts()

BlastProtector       134
CrossCoupling         70
BlastProtector NV      4
CrossCoupling NV       1
Name: type, dtype: int64

In [94]:
BP, CC, BP_NV, CC_NV = features['type'].value_counts()

pie = make_subplots(rows=1, cols=2, specs=[[{"type": "domain"}, {"type": "domain"}]])

pie.add_trace(go.Pie(
     values=[BP, BP_NV],
     labels=['BlastProtector', 'No vertical component'],
     domain=dict(x=[0, 0.5]),
     name="Blast Protector",
     sort=False,
     marker_colors = ['crimson', 'lightgrey']
     ), 
     row=1, col=1)


pie.add_trace(go.Pie(
     values=[CC, CC_NV],
     labels=['CrossCoupling', 'No vertical component'],
     domain=dict(x=[0, 0.5]),
     name="Cross Coupling",
     sort=False,
     marker_colors = ['lightsalmon', 'lightgrey']
     ), 
     row=1, col=2)

# pie.add_trace(go.Pie(
#      values=cc.loc[cc['type_noNV'].isin(['C'])]['type'].value_counts(),
#      labels=cc.loc[cc['type_noNV'].isin(['C'])]['type'].value_counts().index,
#      domain=dict(x=[0, 0.5]),
#      name="Centralizer",
#      sort=False,
#      marker_colors = ['lightblue', 'lightgrey']
#      ), 
#      row=1, col=3)

pie.update_layout(height=600, width=800, title_text="Success rate by jewelry type", template="plotly")
pie.show()

In [ ]:
# fig  = combined.plot(x='AngleFromHighSideClockwiseDegrees', y='depth', kind='scatter', color='type', custom_data=['name', 'perfAngleFromHighSideClockwiseDegrees'], color_discrete_map={'CC':'goldenrod', 'BP':'red', 'C':'lightblue', 'PERFS':'blue', 'NV':'grey'})
# fig.update_layout(hovermode="y", title="Jewelry vs Perfs", legend_title="Type", legend_orientation="h", template="plotly")
# fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
# fig.update_xaxes(range=[-185, 185])
# fig.update_yaxes(autorange="reversed")

In [95]:
pio.write_html(fig, file='plotly/s.html', auto_open=False)
pio.write_html(pie, file='plotly/sc.html', auto_open=False)